In [1]:
import os
import awkward as ak
import numpy as np
import pandas as pd
import uproot
from coffea.nanoevents import NanoEventsFactory
from ecalphisym import EcalPhiSymSchema

In [2]:
import matplotlib.pyplot as plt
import mplhep
%matplotlib inline
plt.rc('font',**{'family':'sans-serif', 'sans-serif':['Helvetica'], 'size':'20'})
plt.rcParams['figure.figsize'] = '10,10'
plt.rcParams['xaxis.labellocation'] = 'right'
plt.rcParams['yaxis.labellocation'] = 'top'
plt.style.use(mplhep.style.ROOT)

In [3]:
# path to W/Z E/p - PhiSym 
runs = NanoEventsFactory.from_root('../Run2018D_test.root',
                                   schemaclass=EcalPhiSymSchema,
                                   treepath="/Runs").events()
eop = uproot.open('../EGamma-Run2018B-ZSkim-17Sep2018-v1-316998-319312.root:selected')

In [5]:
counts = np.unique(runs.EcalPhiSymInfo.fill, return_index=True)[1]
splits = np.diff(np.concatenate([counts, [len(runs.EcalPhiSymInfo.fill)]]))
ebhits = ak.unflatten(runs.EcalPhiSymEB, splits, axis=0, behavior=runs.behavior).sum(axis=1)

In [6]:
ak_eop = eop.arrays(filter_name = "/charge|eta/")

In [7]:
# ieat histo phisym
hist_phisym,bins =np.histogram(ak.to_numpy(ebhits.ieta[1,:]), weights=ak.to_numpy(ebhits.sumet[1,:]), 
           bins=171, range=[-85.5, 85.5])
hist_phisym = np.delete(hist_phisym, 85)
hist_phisym


array([6470558.5, 5913816.5, 5702504.5, 5431461. , 5247154.5, 5215110. ,
       5062798.5, 5117713.5, 5032733. , 4983191.5, 4716124. , 4711281.5,
       4754399. , 4913999.5, 4742408. , 4466268. , 4525836. , 4457386.5,
       4479294.5, 5181097. , 4620260. , 4621286. , 4627073. , 4698097. ,
       4797910. , 5059805. , 5145231. , 5126996.5, 5150545.5, 5151671. ,
       4966093. , 5023423.5, 4994784.5, 5166894. , 5159892.5, 5233955. ,
       5452878.5, 5442860.5, 5442160. , 6299220. , 5239514.5, 5291411.5,
       5193469.5, 5258668.5, 5153777.5, 5265008.5, 5235277.5, 5145459.5,
       5181328. , 5071901. , 5615727. , 4936528.5, 4968877. , 4832296.5,
       4943717.5, 4722284. , 4813020. , 4808355. , 4813853. , 5311121.5,
       4667952. , 4642310. , 4691297.5, 4689237. , 4636837.5, 4534372.5,
       4527644. , 4539809. , 4519824. , 4529935. , 4503815.5, 4630869. ,
       4681728. , 4782128.5, 4791029.5, 4665299. , 4700452.5, 4702800. ,
       4613416.5, 4610139.5, 4844667. , 4908736.5, 

In [8]:
# ieat histo phisym
hist_eop,bins =np.histogram(ak.to_numpy(ak.mask(ak_eop.etaEle[:,0], (ak_eop.chargeEle[:,0]  * ak_eop.chargeEle[:,1]  == 0)  , valid_when=False))/0.0175, bins=171, range=[-85.5, 85.5])
hist_eop = np.delete(hist_eop, 85)
hist_eop

array([ 2694,  9004, 12889, 13427, 13329, 13446, 13573, 13607, 13697,
       13518, 13442, 13635, 13753, 14156, 14386, 14373, 14517, 14803,
       14738, 14078, 12885, 14331, 15477, 15925, 16201, 16279, 16421,
       16567, 16814, 16484, 16686, 16575, 16613, 16616, 17083, 16817,
       16967, 17339, 16885, 16186, 15473, 15879, 16797, 17520, 17487,
       17850, 18179, 18139, 18481, 18409, 18524, 18648, 18654, 18815,
       18420, 18889, 18452, 18387, 18320, 17532, 17325, 17894, 18609,
       19303, 19043, 19089, 18911, 18591, 19184, 19130, 19115, 19095,
       19021, 19139, 19178, 19021, 19283, 19109, 19190, 18955, 19000,
       18947, 18921, 18044, 17027, 16788, 18514, 19694, 19413, 19436,
       19171, 19149, 19126, 19479, 19384, 19595, 19431, 19392, 19351,
       19207, 19202, 19276, 19161, 19258, 19268, 18991, 19217, 18929,
       18434, 17794, 17984, 18999, 18935, 18859, 18756, 18838, 18852,
       18889, 18535, 18689, 18736, 18368, 18448, 18438, 18206, 18093,
       18031, 17936,

In [9]:
w = hist_eop/hist_phisym * (np.sum(hist_phisym) / np.sum(hist_eop) )

In [10]:
# save weights
np.savetxt('weight.txt', w, delimiter=',')   